In [ ]:
# Importar bibliotecas
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Carregar o modelo pré-treinado InceptionV3
base_model = keras.applications.InceptionV3(weights='imagenet', include_top=False)

# Congelar as camadas do modelo base
for layer in base_model.layers:
    layer.trainable = False

# Criar o modelo de classificação personalizado
x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(1024, activation='relu')(x)

predictions = keras.layers.Dense(3, activation='softmax')(x)

model = keras.Model(inputs=base_model.input, outputs=predictions)

# Compilar o modelo
model.compile(optimizer=keras.optimizers.Adam(lr=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Carregar e pré-processar os dados de treinamento e validação
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    '/content/data/train',
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical')

validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(
    '/content/data/validation',
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical')

# Treinar o modelo
model.fit(train_generator, epochs=10, validation_data=validation_generator)


Found 1805 images belonging to 3 classes.
Found 200 images belonging to 3 classes.
Epoch 1/10
57/57 [==============================] - ETA: 0s - loss: 0.1855 - accuracy: 0.9363